In [2]:
import pandas as pd
import numpy as np
import math
import requests

liste_med_ban = pd.read_csv('liste_med_ban.csv')

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_2840/3163695580.py:6: DtypeWarning: Columns (28,29,30,31,32,35,41,42,43,44,45,46,50) have mixed types. Specify dtype option on import or set low_memory=False.
  liste_med_ban = pd.read_csv('liste_med_ban.csv')


In [25]:
def cp_et_nom(cp,nom):
    if isinstance(cp,int) and isinstance(nom,str):
        return (str(cp)+' '+nom)
    else:
        return np.nan

In [26]:
liste_med_ban['cp_ville'] = liste_med_ban.apply(lambda x: cp_et_nom(x['codepostalcoordstructure'],x['libellécommunecoordstructure']),axis=1)
liste_med_ban['cp_ville'].iloc[73370]

'94456 Limeil-Brévannes'

In [ ]:
def transfo_adresse(CodePostal,NumeroVoie,NomVoie):
    num = (int(NumeroVoie))
    return (str(num)+', '+str(NomVoie)+', '+str(CodePostal))

liste_med_ban['adresse'] = liste_med_ban.apply(lambda x: transfo_adresse(x['cp_ville'],x['numérovoiecoordstructure'],x['Voie']),axis=1)

In [27]:
API_URL = "https://api-adresse.data.gouv.fr/{}/"

def req_to(endpoint: str, params: dict | None = None, files: dict | None = None) -> dict:
    url = API_URL.format(endpoint)

    if not files:
        response = requests.get(url, params=params)

    else:
        response = requests.post(url, files=files, data=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception(f"Request failed with code : {response.status_code}")


def get_geocode(**kwargs) -> dict:
    if not 'q' in kwargs:
        raise ValueError('Il manque un argument obligatoire : q')
    
    return req_to("search", kwargs)


def get_reverse(**kwargs) -> dict:
    if not('lat' in kwargs and 'lon' in kwargs):
        raise ValueError('Il manque des arguments obligatoires : lat et lon')
    
    return req_to("reverse", kwargs)


def get_geocode_csv(file_path: str, **kwargs) -> dict:
    with open(file_path, 'rb') as file:
        files = {'data': file}
        response = req_to("search/csv", kwargs, files) 
    
    return response


def get_reverse_csv(file_path: str) -> dict:
    with open(file_path, 'rb') as file:
        files = {'data': file}
        response = req_to("reverse/csv", files=files) 

    return response

In [28]:
numero=str(int(liste_med_ban['numérovoiecoordstructure'].iloc[0]))
voie = liste_med_ban['Voie'].iloc[0]
cp_ville=liste_med_ban['cp_ville'].iloc[0]
adresse = numero+', '+voie+', '+cp_ville
print(adresse)

# Requête simple directe:
res = get_geocode(q = adresse)
print(res)
#print(type(res['properties']),res['properties'][0]['x'])

52, rue jean jaures, 97110 Pointe-à-Pitre
{'type': 'FeatureCollection', 'version': 'draft', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-61.536772, 16.23989]}, 'properties': {'label': '52 Rue Jean Jaures 97110 Pointe-à-Pitre', 'score': 0.9688018181818181, 'housenumber': '52', 'id': '97120_0496_00052', 'banId': '3565ae2f-5cbf-4aab-8543-7a3fda9b9059', 'name': '52 Rue Jean Jaures', 'postcode': '97110', 'citycode': '97120', 'x': 656379.35, 'y': 1796029.48, 'city': 'Pointe-à-Pitre', 'context': '971, Guadeloupe', 'type': 'housenumber', 'importance': 0.65682, 'street': 'Rue Jean Jaures'}}], 'attribution': 'BAN', 'licence': 'ETALAB-2.0', 'query': '52, rue jean jaures, 97110 Pointe-à-Pitre', 'limit': 5}


In [29]:
def recup_long(adr):
    if isinstance(adr,str):
        try:
            return(get_geocode(q=adr)['features'][0]['geometry']['coordinates'][0])
        except (IndexError, KeyError, TypeError):
            print(adr)
            return None
    else :
        return (np.nan)

liste_med_ban['longitude API'] = liste_med_ban['adresse'].apply(recup_long)
#liste_med_ban['latitude API'] = get_geocode(q = liste_med_ban['adresse'])['features'][0]['geometry']['coordinates'][1]

#liste_med_ban['adresse','longitude API','latitude API']

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_98066/3440838157.py", line 17, in <module>
    liste_med_ban['longitude API'] = liste_med_ban['adresse'].apply(recup_long)
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/series.py", line 4924, in apply
    ).apply()
      ^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/apply.py", line 1427, in apply
    return self.apply_standard()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/apply.py", line 1507, in apply_standard
    mapped = obj._map_values(
             ^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11

In [3]:
def distance_long_lat(lat1, lon1, lat2, lon2):
    # Rayon de la Terre en km
    R = 6371.0
    
    # Conversion des degrés en radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Calcul des différences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # Formule de Haversine
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distance
    distance = R * c
    
    return distance

# Exemple d'utilisation
lat1, lon1 = 48.8566, 2.3522  # Paris
lat2, lon2 = 34.0522, -118.2437  # Los Angeles

distance = distance_long_lat(lat1, lon1, lat2, lon2)
print(f"La distance entre Paris et Los Angeles est d'environ {distance:.2f} km.")


La distance entre Paris et Los Angeles est d'environ 9085.51 km.


In [31]:
liste_med_ban['code dépar']

0        971
1        971
2        971
3        971
4        971
        ... 
73369     94
73370     94
73371     93
73372     94
73373     94
Name: code dépar, Length: 73374, dtype: int64

In [32]:
departement_choisi = 39

liste_med_ban_dep=liste_med_ban[liste_med_ban['code dépar']==departement_choisi]
liste_med_ban_dep.shape

(289, 76)

In [35]:
df_cross_0 = pd.read_csv('df_cross_0.csv')

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_2840/1526306788.py:1: DtypeWarning: Columns (100,101,102,103,104,107,113,114,115,116,117,118,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cross_0 = pd.read_csv('df_cross_0.csv')


In [41]:
def distance_bis(x1,y1,x2,y2):
    return((np.sqrt((x1-x2)**2+(y1-y2)**2))/1000)

In [47]:
dico_d={}
df_cross_0['distance']=0
codegeo1 = df_cross_0['codgeo_x'].iloc[0]
Latitude1= df_cross_0['xcl2154_x'].iloc[0]
Longitude1=  df_cross_0['ycl2154_x'].iloc[0]
for i in range(df_cross_0.shape[0]):
    codegeo2 = df_cross_0['codgeo_y'].iloc[i]
    if codegeo1==codegeo2:
        None
    elif (codegeo2,codegeo1) in dico_d :
        df_cross_0['distance'].iloc[i]=dico_d[(codegeo2,codegeo1)]
    elif (codegeo1,codegeo2) in dico_d :
        df_cross_0['distance'].iloc[i]=dico_d[(codegeo1,codegeo2)]
    else :
        distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
        dico_d[(codegeo1,codegeo2)]=distance12
        df_cross_0['distance'].iloc[i]=distance12

df_cross_0['distance'].describe()

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_2840/3280399174.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_cross_0['distance'].iloc[i]=distance12
/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_2840/3280399

count    72481.000000
mean       320.539617
std        220.789479
min          0.000000
25%        130.600829
50%        334.599937
75%        483.182548
max        880.446435
Name: distance, dtype: float64

In [48]:
print(distance_bis(df_cross_0['xcl2154_x'].iloc[0], df_cross_0['ycl2154_x'].iloc[0],df_cross_0['xcl2154_y'].iloc[2], df_cross_0['ycl2154_y'].iloc[2]))
print(df_cross_0['xcl2154_x'].iloc[0], df_cross_0['ycl2154_x'].iloc[0],df_cross_0['xcl2154_y'].iloc[2], df_cross_0['ycl2154_y'].iloc[2])

664.640860630762
652492 6862009 633310 6197645


In [49]:
df_cross_0[['codepostalcoordstructure_x','codepostalcoordstructure_y','distance']]

,codepostalcoordstructure_x,codepostalcoordstructure_y,distance
0,75010,95490,30.998044
1,75010,75010,0.000000
2,75010,11500,664.640861
3,75010,56340,427.248816
4,75010,93100,6.526897
...,...,...,...
72476,75010,94275,4.909933
72477,75010,94456,15.177672
72478,75010,93370,16.796641
72479,75010,94275,4.909933


### API OpenRouteService

In [2]:
import openrouteservice

# Clé API OpenRouteService
client = openrouteservice.Client(key='5b3ce3597851110001cf6248fa09b3e419ef40e182777370274818e9')

# Coordonnées (longitude, latitude) pour le départ et l'arrivée
coordinates = [[2.3522, 48.8566], [4.8357, 45.764]]

# Calcul de l'itinéraire
route = client.directions(coordinates, profile='driving-car')

# Temps de trajet en secondes
duration = route['routes'][0]['summary']['duration']
print(f"Temps de trajet : {duration / 60} minutes")

Temps de trajet : 276.305 minutes
